## 4.2 Data Quality Checks

Data quality checks includes

Data schema of every dimensional table matches data model
No empty table after running ETL data pipeline

In [1]:
import os
import configparser
from pathlib import Path
from pyspark.sql import SparkSession

In [2]:
config = configparser.ConfigParser()
config.read('../config/.env', encoding='utf-8-sig')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']
SOURCE_S3_BUCKET = config['S3']['SOURCE_S3_BUCKET']
DEST_S3_BUCKET = config['S3']['DEST_S3_BUCKET']

In [3]:
spark = SparkSession.builder\
                    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0")\
                    .enableHiveSupport().getOrCreate()

22/10/24 00:53:13 WARN Utils: Your hostname, Renatos-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.5 instead (on interface en0)
22/10/24 00:53:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/opt/homebrew/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/renatomeira/.ivy2/cache
The jars for the packages stored in: /Users/renatomeira/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-80e49499-dd25-454d-ab0d-3fb58ad9fd9d;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;2.7.0 in central
	found org.apache.hadoop#hadoop-common;2.7.0 in central
	found org.apache.hadoop#hadoop-annotations;2.7.0 in central
	found com.google.guava#guava;11.0.2 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found commons-cli#commons-cli;1.2 in central
	found org.apache.commons#commons-math3;3.1.1 in central
	found xmlenc#xmlenc;0.52 in central
	found commons-httpclient#commons-httpclient;3.1 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.4 in central
	found commons-io#commons-io;2.4 in central
	found commons-net#commons-net;3.1 in central
	found commons-collections#com

22/10/24 00:53:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/24 00:53:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### 1. Data schema of every dimensional table matches data model

In [4]:
bucket = Path(DEST_S3_BUCKET)

In [5]:
for dir in bucket.iterdir():
    if dir.is_dir():
        path = str(dir)
        df = spark.read.parquet(path)
        print("Table: " + path.split('/')[-1])
        schema = df.printSchema()

Table: fact_immigration
root
 |-- cic_id: double (nullable = true)
 |-- city_code: string (nullable = true)
 |-- arrive_date: date (nullable = true)
 |-- departure_date: date (nullable = true)
 |-- mode: double (nullable = true)
 |-- visa: double (nullable = true)
 |-- immigration_id: long (nullable = true)
 |-- country: string (nullable = true)
 |-- year: double (nullable = true)
 |-- month: double (nullable = true)
 |-- state_code: string (nullable = true)

Table: dim_immigration_airline
root
 |-- cic_id: double (nullable = true)
 |-- airline: string (nullable = true)
 |-- admin_num: double (nullable = true)
 |-- flight_number: string (nullable = true)
 |-- visa_type: string (nullable = true)
 |-- immigration_airline_id: long (nullable = true)

Table: dim_temperature
root
 |-- date: date (nullable = true)
 |-- avg_temp: string (nullable = true)
 |-- avg_temp_uncertnty: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- latitude: s

### 2. Data quality on table check

In [6]:
for file_dir in bucket.iterdir():
    path = str(file_dir)
    df = spark.read.parquet(path)
    records = df.count()
    table = path.split('/')[-1]
    print(f"Data quality on table {table} check passed with {records} records")

Data quality on table fact_immigration check passed with 3096313 records
Data quality on table dim_immigration_airline check passed with 3096313 records
Data quality on table dim_temperature check passed with 687289 records
Data quality on table dim_demographics check passed with 2891 records
Data quality on table country_code check passed with 288 records
Data quality on table state_code check passed with 54 records
Data quality on table dim_immigration_person check passed with 3096313 records
Data quality on table city_code check passed with 659 records
